<font size = 5>匯入檔案</font>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fileArr = ['../input/pack-problem/p07_c.txt','../input/pack-problem/p07_p.txt','../input/pack-problem/p07_s.txt','../input/pack-problem/p07_w.txt']
container = 0
obj_v = [] #物品對應價值
obj_w = [] #物品對應重量
num = 0 #需告並初始化num 
answer = [] #用來裝讀取進來的正確答案

#讀取相關檔案
for file in fileArr:
    f = open(file)
    if(fileArr.index(file) == 0):
        for line in f.readlines():
            container = int(line)
        f.close
    elif(fileArr.index(file) == 1):
        for line in f.readlines():
            obj_v.append(int(line))
        f.close
    elif(fileArr.index(file) == 2):
        for line in f.readlines():
            answer.append(int(line))
        f.close       
    else:
        for line in f.readlines():
            obj_w.append(int(line))
        f.close 
        
num = len(obj_w) #1.用來產生動態規劃算法的二維陣列之列數。2.用來產生爬山和退火算法之隨機物品拿取組合

<font size=5>動態規劃演算法</font>

In [ ]:
option = [] #背包裝的物品組合
bag = [] #背包裝的所有物品對應價值

def track(table,container,obj_w):
    bag = []
    option = []
    for i in range(len(obj_w), 1, -1): #從最底的列往上找
        if table[i][container] != table[i-1][container]: #代表目前的物品組合為前一組物品組合加上目前可再選的物品
            bag.append(obj_w[i-1])
            container = container - obj_w[i-1]
            option.append(1)
        else: #代表目前的物品組合為前一組物品組合，可能背包塞不下或是前一組物品組合加上可選物品的總價值沒有大於前一組物品組合，因此目前的物品組合就沿用前一組物品組合
            option.append(0)

    #當只有第一個物品可以裝時，無論背包大小多大都只能放這個物品
    if table[1][container] > 0:
        bag.append(obj_w[0])
        option.append(1)

    return bag,option

        
#建立num+1列、container+1欄的全零二為矩陣，其中第零列及第零欄是用來計算第一列及欄所設，表示初始狀態
table = [[0 for j in range(container + 1)] for i in range(num + 1)]
for i in range(1, num + 1):
    for j in range(1, container + 1):
        table[i][j] = table[i - 1][j]
        #先判斷目前背包容量是否放得下當前物品且判斷上一列的背包最高價值加上現在可選的物品價值是否大於現在背包的價值，若是，則代表可以更新現有背包價值。再透過判斷上一列的最高價值
        if j >= obj_w[i-1] and table[i][j] < table[i-1][j-obj_w[i-1]] + obj_v[i-1]:
            table[i][j] = max(table[i-1][j-obj_w[i-1]] + obj_v[i-1], table[i-1][j])

bag,option = track(table,container,obj_w) #將小問題的解組合成大問題的解即為整個問題的解
print('答案:',option[::-1])
if(option[::-1] == answer):
    print('答案比對後相等')

<font size=5>爬山演算法</font>

In [ ]:
import random


def generate_initial_value(num):
    #用來產生所有物品對應的"拿"或"不拿"的組合，1代表拿，0代表不拿
    initOption = []
    for i in range(num):
        gene=random.randint(0,1)
        initOption.append(gene)
    return initOption

def score_weight(initial,obj_w):
    #透過將物品重量乘上產生的物品組合來產生已拿取的物品總重量
    weightlist=[]
    for i in range(len(initial)):
        score = obj_w[i]*initial[i]
        weightlist.append(score)
    return sum(weightlist)

def score_value(initial,obj_v):
    #透過將物品價值乘上產生的物品組合來產生已拿取的物品總價值
    valuelist=[]
    for i in range(len(initial)):
        score = obj_v[i]*initial[i]
        valuelist.append(score)
    return sum(valuelist)

def is_feasible(weight,container):
    #用來判斷目前裝的物品重量是否小於背包可承受重量，若大於，代表此次迭代所產生的物品組合之總重量超過背包可承受重量，因此逕行下次迭代以求出物品組合之重量為背包可承受重量
    if weight < container:
        return True
    return False
        
def main(n):
    optimal = 0 #起始背包最高價值(初始背包最高價值為0)，每一次迭代如有最高價值大於上一次的最高價值，則optimal儲存此次最高價值
    best_binary=[] #起始物品拿取組合，也就是最優解，每一次迭代如有最優解大於上一次最優解，則best_binary儲存此次物品組合
    for i in range(n):
        initial = generate_initial_value(num) #用來隨機產生物品拿取的組合
        value = score_value(initial,obj_v) #計算目前拿取組合的物品總價值
        weight = score_weight(initial,obj_w) #計算目前拿取組合的物品總重量
        plt.plot(i,optimal,marker='_',color = 'b',linewidth=0.1,linestyle='-')
        if is_feasible(weight,container): #判斷目前裝的物品重量是否小於背包可承受重量，接著再判斷目前最優解(目前所裝物品總價值)
            if value > optimal: #判斷目前背包價值是否大於上一次迭代的背包價值，若有，則將此次背包價值儲存，並用來比較下次迭代的背包價值
                optimal = value
                best_binary = initial #目前隨機拿取的物品是目前的最佳拿取組合
                best_weight = weight #目前最優解的物品總重量
                print("目前最優解(最高價值)，",optimal,"在迭代:",i,
                "物品組合",best_binary,"物品總重:",best_weight)
            
    plt.show()    
    return optimal,"物品組合:",best_binary,"的總重量:",best_weight,"是否符合正確答案:",best_binary == answer
#答案有時不正確，因為可能陷入局部最優解，無法繼續往前迭代找到更優解
print("最終優解(最終價值):",main(500))



<font size=5>模擬退火演算法</font>

In [ ]:
import random
import math
global time,balance;  #time為迭代次數，balance為平衡次數
global best,T,af;   #best為全局最優解，T為温度，af為退火率
m=10; T=200.0; af =0.95
time =500;  balance = 10; 
best_way=[0]*num   
now_way=[0]*num  #best_way為全局最優解之物品組合，now_way為目前最優解

def cop(a,b,le):  #複製b的內容給a
    for i in range(le):
        a[i]=b[i]
def calc(x): #計算背包價值
    global container,wsum
    vsum=0
    wsum=0
    for i in range(num):
        vsum += x[i]*obj_v[i] 
        wsum += x[i]*obj_w[i];    
    return  vsum
def produce(): #產生初始隨機物品組合
    while (1>0):
        for k in range(num):
            if(random.random() < 0.5):  
                now_way[k]=1
            else: 
                now_way[k]=0
        calc(now_way)
        if(wsum <container): 
            break
    global best
    best=calc(now_way)
    cop(best_way,now_way,num)

def init(): #初始化函数
    global best
    best=-1
    produce() #用來產生初始隨機物品組合
def get(x):  #隨機將背包已存在物品拿出
    while(1>0):
        ob = random.randint(0,num-1)
        if(x[ob]==1): 
            x[ob]=0
            break
def put(x):  #隨機放入背包中不存在的物品
    while(1>0):
        ob = random.randint(0,num-1)
        if(x[ob]==0): 
            x[ob]=1
            break      
def main(): 
    global best,T,balance
    test=[0]*num
    now = 0   #目前的迭代的背包價值
    for i in range(balance):
        now = calc(now_way)
        cop(test,now_way,num)
        ob = random.randint(0,num-1); #隨機選取某個物品
        if(test[ob]==1): 
            put(test)
            test[ob]=0  #在背包中則將其拿出，並加入其它物品
        else:   #不在背包中則直接加入或替換掉已在背包中的物品
            if(random.random()<0.5):
                test[ob]=1 
            else: 
                get(test) 
                test[ob]=1
        temp = calc(test)
        if(wsum > container): #若此迭代的物品組合重量大於背包可承受之重則逕行下次迭代
            continue  
        if(temp > best): 
            best=temp 
            cop(best_way,test,num) #更新全局最優解
        
        if(temp > now): 
            cop(now_way,test,num) #直接接受新解
        else:
            g = 1.0*(temp-now)/T
            if(random.random() < math.exp(g)): #有機率接受較差解，指數函數之指數愈小，則函數值愈小，因此接受較差解機率愈小
                cop(now_way,test,num) #接受較差解   
        
init()
isGood = 0
for i in range(time):      
    main()
    plt.plot(i,best,marker='_',color = 'b',linewidth=0.1,linestyle='-')
    T = T*af  #每一次迭代都要降溫
    if(best_way==answer):  
        print('最優解,迭代',i+1); isGood = 1  
        break  #找到全局最優解直接退出迭代，因為此演算法相較爬山演算法，擁有機率接受較差解的特性，就有可能忽略最優解而去找較差解
plt.show()    
if(isGood == 0):   
    print('次優解:',best,'迭代',time)
print('物品組合：',best_way)